# JSON exercise

Download the data from [**here**](https://drive.google.com/file/d/1DGaX5AVfYhmWeb15lI-MzUbSKTYSz9fQ/view?usp=sharing) and answer following questions:
1. Find the 10 countries with most projects
2. What are top 10 sectors with projects?
3. Find the top 10 major project themes (using column 'mjtheme_namecode')
4. In 3. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [1]:
import pandas as pd
import json


In [2]:
# Read Json File:
import json
#load json object
with open('world_bank_projects.json') as f:
    nested_json = json.load(f)
# print(nested_json)
print(type(nested_json))

<class 'list'>


In [3]:
type(nested_json)



list

In [4]:
# 1. Find the 10 countries with most projects
df =    pd.DataFrame(nested_json)
df.groupby('countryname')['project_name'].count().sort_values(ascending=False).head(10)

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Nepal                              12
Kingdom of Morocco                 12
Republic of Mozambique             11
Africa                             11
Name: project_name, dtype: int64

In [5]:
# 2. What are top 10 sectors with projects?

# expand sector:
expand_sector = pd.json_normalize(data=nested_json, record_path='sector', meta=['project_name'])
# type(expand_sector)
expand_sector.groupby('Name')['project_name'].count().sort_values(ascending=False).head(10)

Name
Other social services                                       106
Central government administration                            82
Sub-national government administration                       75
General agriculture, fishing and forestry sector             65
Health                                                       63
General public administration sector                         51
Rural and Inter-Urban Roads and Highways                     44
General water, sanitation and flood protection sector        39
Agricultural extension and research                          38
Public administration- Agriculture, fishing and forestry     36
Name: project_name, dtype: int64

In [6]:
# 3. Find the top 10 major project themes (using column 'mjtheme_namecode') 
expand_mjtheme = pd.json_normalize(data=nested_json, record_path='mjtheme_namecode', meta=['project_name'])

expand_mjtheme.groupby('code')['project_name'].count().sort_values(ascending=False).head(10)


code
11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
Name: project_name, dtype: int64

In [7]:
# 4. In 3. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.
expand_mjtheme_filled = expand_mjtheme.copy()

# expand_mjtheme_filled.sort_values(by = 'code')['name'].replace('', np.nan).fillna(method='ffill')
expand_mjtheme_filled.sort_values(by = 'code')['name'].replace('', method='ffill', inplace=True)



In [8]:
# get a mapping of code to name
# code_to_name = expand_mjtheme.drop_duplicates(subset='code', keep='first').set_index('code')['name'].to_dict()
code_to_name = expand_mjtheme.sort_values(by = 'name', ascending = False).drop_duplicates(subset='code', keep='first').set_index('code')['name'].to_dict()

In [9]:
# show the mapping dictionary we are going to use
code_to_name

{'9': 'Urban development',
 '5': 'Trade and integration',
 '6': 'Social protection and risk management',
 '7': 'Social dev/gender/inclusion',
 '10': 'Rural development',
 '3': 'Rule of law',
 '2': 'Public sector governance',
 '8': 'Human development',
 '4': 'Financial and private sector development',
 '11': 'Environment and natural resources management',
 '1': 'Economic management'}

In [10]:
# fill the empty values with the correct values from mapping
expand_mjtheme_filled['name'] = expand_mjtheme_filled['code'].map(code_to_name)